In [1]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import random
import gzip
import math
import sklearn
import pandas
import ast
import scipy
import torchfm
# from torchfm.model.fm import FactorizationMachineModel
# import torch
import myfm
import scipy.sparse as sps
import json
import ast

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
filePath = "recipes.csv"
recipes = pandas.read_csv(filePath)
# all_tags = {}

recipesDataset = {}
for _, row in recipes.iterrows():
    recipesDataset[row["id"]] = {
        "name": row["name"],
        "recipe_id": row["id"],
        "minutes": row["minutes"],
        "contributor_id": row["contributor_id"],
        "submitted": row["submitted"],
        "tags": row["tags"],
        "nutrition": row["nutrition"],
        "n_steps": row["n_steps"],
        "steps": row["steps"],
        "description": row["description"],
        "ingredients": row["ingredients"],
        "n_ingredients": row["n_ingredients"],
    }
    # recipe_tags = ast.literal_eval(row["tags"])
    # for tag in recipe_tags:
    #     if tag not in all_tags:
    #         all_tags[tag] = len(all_tags)

In [4]:
print(json.dumps(recipesDataset[next(iter(recipesDataset))], indent=4))

{
    "name": "arriba   baked winter squash mexican style",
    "recipe_id": 137739,
    "minutes": 55,
    "contributor_id": 47892,
    "submitted": "2005-09-16",
    "tags": "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
    "nutrition": "[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",
    "n_steps": 11,
    "steps": "['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until

In [5]:
filePath = "interactions.csv"
interactions = pandas.read_csv(filePath)

interactionsDataset = []
for _, row in interactions.iterrows():
    interactionsDataset.append({
        "user_id": row["user_id"],
        "recipe_id": row["recipe_id"],
        "date": row["date"],
        "rating": row["rating"],
        "review": row["review"],
    })

In [6]:
print(json.dumps(interactionsDataset[0], indent=4))

{
    "user_id": 38094,
    "recipe_id": 40893,
    "date": "2003-02-17",
    "rating": 4,
    "review": "Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks."
}


In [7]:
userIDs,itemIDs = {},{}

for d in interactionsDataset:
    u,i = d['user_id'],d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [8]:
nUsers,nItems

(226570, 231637)

In [9]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [10]:
tags_desc = pandas.read_csv('tags_desc.csv')
tags_encoding = {}

i = 0
for _, row in tags_desc.iterrows():
    tags_encoding[row['tag']] = i
    i += 1

tags_asc = pandas.read_csv('tags_asc.csv')
for _, row in tags_asc.iterrows():
    tags_encoding[row['tag']] = i
    i += 1

In [ ]:
X = []
y = []
features = []
for d in interactionsDataset:
    u = d['user_id']
    i = d['recipe_id']
    steps = recipesDataset[i]['n_steps']
    sugar = json.loads(recipesDataset[i]['nutrition'])[2]
    protein = json.loads(recipesDataset[i]['nutrition'])[4]
    recipe_tags = ast.literal_eval(recipesDataset[i]["tags"])
    row = []
    # if steps <= 30:     #n_steps: 0-30, 31-50, 51+
    #     row += [1,0,0]
    # elif steps <= 50:
    #     row += [0,1,0]
    # else:
    #     row += [0,0,1]

    # protein_bucket = int(protein / 5) if protein < 50 else 9
    # proteins = [0] * 10
    # proteins[protein_bucket] = 1
    # row += proteins

    sugar_bucket = int(sugar / 5) if sugar < 100 else 19
    sugars = [0] * 20
    sugars[sugar_bucket] = 1
    row += sugars

    # tags = [0] * len(all_tags)
    # for tag in recipe_tags:
    #     tags[all_tags[tag]] = 1
    # row += tags

    tags = [0] * len(tags_encoding)
    for tag in recipe_tags:
        if tag in tags_encoding:
            tags[tags_encoding[tag]] = 1
    row += tags
    features.append(row) 
    X.append([userIDs[u], itemIDs[i]])
    y.append(d['rating'])
split = int(len(X)*0.9)

In [21]:
Xtrain = ohe.fit_transform(np.array(X[:split]))
ytrain = np.array(y[:split])
Xtest = ohe.transform(np.array(X[split:]))
ytest = np.array(y[split:])
features_train = np.array(features[:split])
features_test = np.array(features[split:])

In [22]:
Xtrain.shape

(1019130, 418002)

In [23]:
X_train = sps.hstack([Xtrain, features_train])
X_test = sps.hstack([Xtest, features_test])

In [ ]:
fm = myfm.MyFMRegressor(rank=5)

In [31]:
fm.fit(X_train, ytrain, n_iter=100)

alpha = 1.07 w0 = 4.11 : 100%|██████████| 100/100 [00:46<00:00,  2.15it/s]


In [32]:
prediction = fm.predict(X_test)

In [33]:
prediction

array([4.24829565, 4.7609565 , 4.78880742, ..., 4.58918249, 4.59242912,
       4.17245185])

In [34]:
print(((ytest - prediction)**2).mean()**.5)

1.1891012220922386


rank 1, 80/20, 1000 iter, MSE: 1.1997331904645177
rank 2, 90/10, 100 iter, MSE: 1.1892600032936387
rank 1, 90/10, 100 iter, MSE: 1.1891660151304382
rank 5, 90/10, 100 iter, MSE: 1.189063754787245
rank 3, 90/10, 100 iter, MSE: 1.1891654617759837
rank 5, 90/10, 100 iter, n_steps, MSE: 1.1896700940789633
rank 5, 90/10, 1000 iter, n_steps, MSE: 1.1890897607570092
rank 3, 90/10, 100 iter, n_steps, MSE: 1.188501736141432
rank 2, 90/10, 100 iter, n_steps, MSE: 1.1890215516655562
rank 4, 90/10, 100 iter, n_steps, MSE: 1.1898952933103657
rank 1, 90/10, 100 iter, n_steps, MSE: 1.1893514310273694
rank 5, 90/10, 100 iter, sugar, MSE: 1.1885092216834368
rank 3, 90/10, 100 iter, sugar, MSE: 1.188272832598858
rank 4, 90/10, 100 iter, sugar, MSE: 1.1878161153746627
rank 2, 90/10, 100 iter, sugar, MSE: 1.1880467187381674
rank 6, 90/10, 100 iter, sugar, MSE: 1.188239250596974
rank 5, 90/10, 100 iter, n_steps, sugar, MSE: 1.1917040793538252
rank 3, 90/10, 100 iter, n_steps, sugar, MSE: 1.1895674636726772
rank 4, 90/10, 100 iter, n_steps, sugar, MSE: 1.1897596852091208
rank 5, 90/10, 100 iter, tags, MSE: 1.1890169239678001
rank 3, 90/10, 100 iter, tags, MSE: 1.19212364114989
rank 4, 90/10, 100 iter, tags, MSE: 1.1898348454922785
rank 5, 90/10, 100 iter, sugar, tags, MSE 1.1887483367286866
rank 3, 90/10, 100 iter, sugar, tags, MSE: 1.18968113283392
rank 4, 90/10, 100 iter, sugar, tags, MSE: 1.1904847477649545
rank 5, 90/10, 100 iter, sugar, limited_tags, MSE: 1.1869605050351804
rank 4, 90/10, 100 iter, sugar, limited_tags, MSE: 1.1871103041220747
rank 3, 90/10, 100 iter, sugar, limited_tags, MSE: 1.187576596016662
rank 6, 90/10, 100 iter, sugar, limited_tags, MSE: 1.1875879363862554
rank 5, 90/10, 100 iter, limited_tags, MSE: 1.1888888154188257
rank 4, 90/10, 100 iter, limited_tags, MSE: 1.1887386814606198
rank 3, 90/10, 100 iter, limited_tags, MSE: 1.1891012220922386
rank 5, 90/10, 100 iter, n_steps, sugar, limited_tags, MSE: 1.1905230684773915
rank 5, 90/10, 100 iter, protein, MSE: 1.188244128228723
rank 3, 90/10, 100 iter, protein, MSE: 1.1884093586673925
rank 5, 90/10, 100 iter, sugar, protein, MSE: 1.1874592317739308
rank 4, 90/10, 100 iter, sugar, protein, MSE: 1.1881269345816998
rank 3, 90/10, 100 iter, sugar, protein, MSE: 1.1890019773684568
rank 5, 90/10, 100 iter, protein, limited_tags, MSE: 1.1875904309334224
rank 4, 90/10, 100 iter, protein, limited_tags, MSE: 1.1879463496242961
rank 3, 90/10, 100 iter, protein, limited_tags, MSE: 1.1875456374580644
rank 5, 90/10, 100 iter, sugar, protein, limited_tags, MSE: 1.188361885740543

In [ ]:
# model = fm.run(X[:split], y[:split], X[split:], y[split:])

In [ ]:
#!jupyter nbconvert --to python assignment2.ipynb

[NbConvertApp] Converting notebook assignment2.ipynb to python
[NbConvertApp] Writing 1515 bytes to assignment2.py
